Last Edit: 03/11/2023

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
using Printf
using JSON
using BSON

In [54]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   
param3b = SailorMoon.dynamics_parameters()
plotly()

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Plots.PlotlyBackend()

In [3]:
params1 = [param3b.mu2, param3b.mus, param3b.as, 0.0, param3b.oml, 0.0, 0.0, 0.0]    

params2 = [param3b.mu2, param3b.mus, param3b.as, 0.0, param3b.oml, 0.0, 0.0, 0.0]; 
@printf("%3f",  param3b.lstar)
println("TU: ", param3b.tstar)

384748.322930TU: 375700.3437894195


### Propagate the NRHO from Horizon database 
なんか上手くいってない...?

In [34]:
# (almost) L2-N 4:1 NRHO initial condition from Horizon ()
LU = 389703
TU = 382981

u0 = [
    1.0337146782336497e+0,
    0.0000000000000000e+0,
    1.8913368328700852e-1,
    -3.9574037159804111e-14,
    -1.2736076790606726e-1,
    8.5568227082711327e-13
    ]

u0 = [
    1.0272, 0.0, -0.1906, 0.0, -0.1259, 0.0
]

u0[1:3] = u0[1:3] * LU / param3b.lstar
u0[4:6] = u0[4:6] * (LU / param3b.lstar) / (TU / param3b.tstar)

tof = 4.5

# prob_EMrot = ODEProblem(SailorMoon.rhs_bcr4bp!, u0, [0, tof], params1)
prob_EMrot = ODEProblem(SailorMoon.rhs_cr3bp!, u0, [0, tof], params1)
sol = solve(prob_EMrot, Tsit5(), reltol=1e-12, abstol=1e-12);

In [39]:
pcart = plot3d(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false)

scatter!(pcart, [1.0],[0.0],[0.0], markershape=:diamond,label="Moon")
scatter!(pcart, [1.15568217],[0.0],[0.0])
# scatter!(pcart, [sol[1,1]], [sol[2,1]], [sol[3,1]])
plot!(pcart, sol[1,:], sol[2,:], sol[3,:], linewidth=1.5, label="NRHO")
pcart

### ok then let's make a Halo orbit... 

In [6]:
lp = 2; Az_km = 13970.0
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, alg=Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

Unrecognized keyword arguments: 

┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s

[:method]



┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Linear stability ν = 659.5897456934673


In [33]:
prob_EMrot_manual = remake(prob_EMrot, u0=res.x0, t=[0, tof], p=params1);
println("s0(EMrot):", res.x0)

s0(EMrot):[1.13074045596174, 0.0, 1.1678839182764198e-30, 0.0, 0.12651468961184018, 0.0]


In [23]:
pcart2 = plot3d(size=(700,500), zlim=(-1,1), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false)

scatter!(pcart2, [1.0],[0.0],[0.0],label="Moon")
scatter!(pcart2, [1.15568217],[0.0],[0.0], markershape=:diamond, label="L2")
scatter!(pcart2, [sol[1,1]], [sol[2,1]], [sol[3,1]])
plot!(pcart2, sol[1,:], sol[2,:], sol[3,:], linewidth=1.5, label="NRHO")


### Anyways, transform this coordinate into the SB1 frame and propagate the orbit

In [45]:
u0_SB1 = SailorMoon.transform_EMrot_to_SunB1(res.x0, 0.0, param3b.oml, param3b.as, param3b.mus)
u0_    = SailorMoon.transform_SunB1_to_EMrot(u0_SB1, 0.0, param3b.oml, param3b.as, param3b.mus)

println("original u0 (EMrot): ", res.x0)
println("SB1 frame: ", u0_SB1)
println("original u0 (EMrot): ", u0_)


original u0 (EMrot): [1.13074045596174, 0.0, 1.1678839182764198e-30, 0.0, 0.12651468961184018, 0.0]
SB1 frame: [389.9507969340323, 0.0, 1.1678839182764198e-30, 0.0, 1.1726757587937295, 0.0]
original u0 (EMrot): [2.2157000672988096, 0.0, 1.1678839182764198e-30, 0.0, 1.1726757587937295, 0.0]


In [46]:
prob_SB1rot = ODEProblem(SailorMoon.rhs_bcr4bp!, u0_SB1, [0, tof], params2)
sol = solve(prob_SB1rot, Tsit5(), reltol=1e-12, abstol=1e-12);

In [47]:
pcart3 = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false)
plot!(pcart3, sol[1,:], sol[2,:], linewidth=1.5)

pcart3

### Let's propagate a good initial guess in BCR4BP...

In [12]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [50]:
# S-B1 frame coordinate
s0 = [389.9439269, 2.79E-07, 5.60E-23, -1.03E-06, 1.203809979, 3.76E-31];
s0 = vcat(s0, 1)

7-element Vector{Float64}:
 389.9439269
   2.79e-7
   5.6e-23
  -1.03e-6
   1.203809979
   3.76e-31
   1.0

In [14]:
param = [param3b.mu2, param3b.mus, param3b.as, 0.0, param3b.oml, param3b.omb, 1.0, 0.0, 0.0, 0.0, 0.00, SailorMoon.dv_sun_dir_angles2]
prob_SB1rot = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2_thrust!, s0, [0, 24.5], param)
sol = solve(prob_SB1rot, Tsit5(), reltol=1e-12, abstol=1e-12);

In [15]:
pcart4 = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false)
plot!(pcart4, sol[1,:], sol[2,:], linewidth=1.5)
scatter!(pcart4, [param3b.mus/(param3b.mus+1)*param3b.as],[0.0])
circle = plot_circle(1,param3b.mus/(param3b.mus+1)*param3b.as, 0.0)
plot!(pcart4, circle[1,:], circle[2,:], linewidth=1.5)

In [55]:
s0_EMrot = SailorMoon.transform_SunB1_to_EMrot(s0[1:6], 0.0, param3b.oml, param3b.as, param3b.mus)
println("s0 (SB1): ", s0[1:6])
println("s0 (EMrot): ",s0_EMrot)

C:[1.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.9251999994040079 0.0 1.0 0.0 0.0; -0.9251999994040079 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0]
s0 (SB1): [389.9439269, 2.79e-7, 5.6e-23, -1.03e-6, 1.203809979, 3.76e-31]
s0 (EMrot): [1.123870421929439, 2.79e-7, 5.6e-23, -7.718692001662819e-7, 0.16400506530070103, 3.76e-31]


In [56]:
# prob_EMrot = ODEProblem(SailorMoon.rhs_bcr4bp!, u0, [0, tof], params1)
prob_EMrot = ODEProblem(SailorMoon.rhs_bcr4bp!, s0_EMrot, [0, tof], params1)
sol = solve(prob_EMrot, Tsit5(), reltol=1e-12, abstol=1e-12);

In [60]:
pcart5 = plot(size=(700,500),frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false)

# scatter!(pcart5, [1.0],[0.0],[0.0])
# scatter!(pcart5, [1.15568217],[0.0],[0.0])
# scatter!(pcart5, [sol[1,1]], [sol[2,1]], [sol[3,1]])
plot!(pcart5, sol[1,:], sol[2,:], linewidth=1.5, label="NRHO")
pcart5

In [27]:
param3b.omb

0.07480000059599223

In [29]:
1.99e-7 * param3b.tstar

0.07476436841409448

In [30]:
param3b.oms

-0.9251999994040079